In [3]:
import glob
import json
import pandas as pd
import plotly.graph_objects as go

#pd.set_option('display.max_rows', None)

SKLEARN_DATA = "../data/sklearn/modules/sklearn_modules.json"

ALL_SKLEARN_PROJECTS = "statistics/sklearn/statistics/*"
ALL_TENSORFLOW_PROJECTS = "statistics/tensorflow/statistics/*"
ALL_PYTORCH_PROJECTS = "statistics/pytorch/statistics/*"
SKLEARN_RESULTS = "statistics/sklearn/results/"

Calculate number of options for each project

In [4]:
projects = []
project_names = []
project_total_default = []
project_total_custom = []

with open(SKLEARN_DATA, "r", encoding="utf-8") as sklearn_file:
    sklearn_data = json.load(sklearn_file)

for project in glob.glob(ALL_SKLEARN_PROJECTS):
    default_counter = 0
    custom_counter = 0
    total_counter = 0
    project_name = project.split("\\")[-1].split(".")[0]
    project_name = project_name.replace("statistics_", "")

    with open(project, "r", encoding="utf-8") as project_file:
        project_data = json.load(project_file)

    for file in project_data:
        file_data = project_data[file]

        for module in file_data:
            module_name = module.split("_")[0]
            sklearn_module = next(filter(lambda x: x["name"] == module_name, sklearn_data))
            module_data = file_data[module]

            if module_data:
                for option in module_data:
                    total_counter += 1
                    try:
                        param = next(filter(lambda x: x[0] == option, sklearn_module["params"]))
                        default_value = param[1].split("=")[-1]
                        option_value = module_data[option]   
                        if default_value == option_value:
                            default_counter += 1
                        else:
                            custom_counter +=1
                    except:
                        custom_counter +=1
    
    percent_default = 0 if total_counter == 0 else round((default_counter/total_counter)*100, 1)
    percent_custom = 0 if total_counter == 0 else round((custom_counter/total_counter)*100, 1)


    project_names.append(project_name)
    project_total_default.append(default_counter)
    project_total_custom.append(custom_counter)

    data = {
        "options_used": {project_name: total_counter},
        "default_options_used": {project_name: default_counter},
        "custom_options_used": {project_name: custom_counter},
        "%_default_options_used": {project_name: percent_default},
        "%_custom_options_used": {project_name: percent_custom},
    }

    df = pd.DataFrame.from_dict(data)
    projects.append(df)

result = pd.concat(projects)
result

,options_used,default_options_used,custom_options_used,%_default_options_used,%_custom_options_used
100DaysofMLCodeChallenge,52,15,37,28.8,71.2
ade-code,0,0,0,0.0,0.0
Adversarial-Robust-Supervised-Sparse-Coding,0,0,0,0.0,0.0
AI-Feynman,6,0,6,0.0,100.0
AMINN,3,0,3,0.0,100.0
...,...,...,...,...,...
TuiGAN-PyTorch,0,0,0,0.0,0.0
VGG_Paper,0,0,0,0.0,0.0
VLight,0,0,0,0.0,0.0
vln-bert,0,0,0,0.0,0.0


In [5]:

fig = go.Figure(data=[
    go.Bar(name='default', x=project_names, y=project_total_default),
    go.Bar(name='custom', x=project_names, y=project_total_custom)
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [6]:

df_without_ml_algorithms = result[result['options_used'] == 0]
len(df_without_ml_algorithms)
df_without_ml_algorithms

,options_used,default_options_used,custom_options_used,%_default_options_used,%_custom_options_used
ade-code,0,0,0,0.0,0.0
Adversarial-Robust-Supervised-Sparse-Coding,0,0,0,0.0,0.0
amortized-rsa,0,0,0,0.0,0.0
ApproPO,0,0,0,0.0,0.0
ATRank,0,0,0,0.0,0.0
...,...,...,...,...,...
TuiGAN-PyTorch,0,0,0,0.0,0.0
VGG_Paper,0,0,0,0.0,0.0
VLight,0,0,0,0.0,0.0
vln-bert,0,0,0,0.0,0.0


How many projects do not use ML algorithms?

In [7]:
df_with_ml_algorithms= result[result['options_used'] > 0]
len(df_with_ml_algorithms)
df_with_ml_algorithms

,options_used,default_options_used,custom_options_used,%_default_options_used,%_custom_options_used
100DaysofMLCodeChallenge,52,15,37,28.8,71.2
AI-Feynman,6,0,6,0.0,100.0
AMINN,3,0,3,0.0,100.0
AttentionXML,7,0,7,0.0,100.0
CE3,2,0,2,0.0,100.0
...,...,...,...,...,...
this-before-that,6,0,6,0.0,100.0
TOCCA,10,1,9,10.0,90.0
trackrcnn_with_deepsort,12,1,11,8.3,91.7
TransformCodingInference,6,0,6,0.0,100.0


In [8]:
print(len(result))
print("Total: ", result["options_used"].sum())
print("Default: ", result["default_options_used"].sum())

158
Total:  759
Default:  160


In [9]:
default_used = df_with_ml_algorithms["%_default_options_used"].sum() / 184
custom_used = df_with_ml_algorithms["%_custom_options_used"].sum() / 184

print("Average default used: ", default_used)
print("Average custom used: ", custom_used)


Average default used:  6.597282608695651
Average custom used:  36.3375


Calculate how many projects use ML algorithms

In [13]:
with open(SKLEARN_DATA, "r", encoding="utf-8") as sklearn_file:
    sklearn_data = json.load(sklearn_file)

projects_with_algorithms = []
projects_without_algorithms = []


for project in glob.glob(ALL_SKLEARN_PROJECTS):
    contains_ml_algorithm = False
    project_name = project.split("\\")[-1].split(".")[0]
    project_name = project_name.replace("statistics_", "")

    with open(project, "r", encoding="utf-8") as project_file:
        project_data = json.load(project_file)

    for file in project_data:
        file_data = project_data[file]

        for module in file_data:
            module_name = module.split("_")[0]
            sklearn_module = next(filter(lambda x: x["name"] == module_name, sklearn_data))
            contains_ml_algorithm = True

        if contains_ml_algorithm:
            projects_with_algorithms.append(project_name)
            break

    if not contains_ml_algorithm:
        projects_without_algorithms.append(project_name)


print("Projects with ML algorithms: ", len(projects_with_algorithms))
print("Projects without ML algorithms: ",len(projects_without_algorithms))
print("Total projects number: ",len(projects_with_algorithms) + len(projects_without_algorithms))

with open(SKLEARN_RESULTS + "projects_without_mla.txt", "w", encoding="utf-8") as source:
    for project in projects_without_algorithms:
        source.write(project + "\n")

            

Projects with ML algorithms:  86
Projects without ML algorithms:  72
Total projects number:  158
ade-code
Adversarial-Robust-Supervised-Sparse-Coding
amortized-rsa
ApproPO
ATRank
attention_network_experiments
Audio_recognition_
AVATAR
baller2vec
bert-ada
BERT_LRP
Brain-Tumour-Segmentation-Dissertation
BrainRegressorCNN
bsldict
cartography
CC-WGAN-GP
chinese-sentiment-classification
cnn-rnf
COVID-DA
CVPR19_Incremental_Learning
DenoiSeg
density-peaks-sentence-clustering
differential-mdl-change-statistics
ethics
etnlp
Fair-Max-Entropy-Distributions
FEDL_pytorch
fever2-columbia
geostyle
GL-AT
GPV
Heterogeneity-Loss-to-Handle-Intersubject-and-Intrasubject-Variability-in-Cancer
HSRL
Image-Interpolation-via-adaptive-separable-convolution
Inductive-representation-learning-on-temporal-graphs
Information-in-Language
IterAvg
key-protected-classification
knee-cartilage-segmentation
lio
MACH
MAN-PAR-
MI-on-EL
mmdetection
multiscale-adversarial-attention-gates
multiview-subword-regularization
Neural